In [0]:
# Data download and unzipping for stage2
%%capture
!pip install kaggle
!mv kaggle.json .kaggle/
!kaggle competitions download -c data-science-bowl-2018 -f stage2_test_final.zip

!mkdir stage2_test_final
!unzip .kaggle/competitions/data-science-bowl-2018/stage2_test_final.zip -d stage2_test_final/

In [0]:
import os
import sys
import warnings
import numpy as np
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from keras.utils import Progbar
from keras.models import load_model
from model import get_unet, iou_metric, iou_metric_batch, my_iou_metric
warnings.filterwarnings('ignore', category=UserWarning, module='skimage')

SEED = 42
np.random.seed(SEED)


TEST_PATH = 'stage2_test_final/'
test_ids = next(os.walk(TEST_PATH))[1]

b = Progbar(len(test_ids))

X_test = np.zeros((len(test_ids), 256, 256, 3), dtype=np.uint8)
sizes_test = []
for n, id_ in enumerate(test_ids):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')
    if len(img.shape) >= 3:
        img = img[:, :, :3]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (256, 256), mode='constant', preserve_range=True)
    X_test[n] = img if len(img.shape) >=3 else img[:,:,np.newaxis]
    b.update(n)


def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1: run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths


# Threshold = 0.5
def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)


# Iterate over the test IDs and generate run-length encodings for each separate mask identified by skimage

def mask_to_rle(preds_test_upsampled):
    new_test_ids = []
    rles = []
    for n, id_ in enumerate(test_ids):
        rle = list(prob_to_rles(preds_test_upsampled[n]))
        rles.extend(rle)
        new_test_ids.extend([id_] * len(rle))
        b.update(n)
    return new_test_ids, rles

Using TensorFlow backend.


3019
3018/3019 [============================>.] - ETA: 0s

In [0]:
best_model = load_model('best_aug_model.h5', custom_objects={'my_iou_metric': my_iou_metric})
stage2_test_pred = best_model.predict(X_test, verbose=1)


3019/3019 [==============================] - 25s 8ms/step


In [0]:
# Create list of upsampled test masks
test_mask_upsampled = []
for i in range(len(stage2_test_pred)):
    test_mask_upsampled.append(resize(np.squeeze(stage2_test_pred[i]),
                                       (sizes_test[i][0],sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

In [0]:
new_test_ids,rles = mask_to_rle(test_mask_upsampled)

In [0]:
# Create submission DataFrame
import pandas as pd
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))

sub.to_csv('sub-dsbowl2018.csv', index=False)